### Part1

In [1]:
import numpy as np

In [2]:
def simplex_algorithm(c1, c2, a11, a12, a21, a22, b1, b2):
    
    c = np.array([c1, c2])
    A = np.array([[a11, a12], [a21, a22]])
    b = np.array([b1, b2])

    num_vars = 2

    num_constraints = 2

    tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))

    tableau[0, :num_vars] = -c  

    tableau[1:, :num_vars] = A
    tableau[1:, -1] = b

    tableau[1, num_vars] = 1
    tableau[2, num_vars + 1] = 1

    while True:
        pivot_col = np.argmin(tableau[0, :-1])
        if tableau[0, pivot_col] >= 0:
            break  

        ratios = tableau[1:, -1] / tableau[1:, pivot_col]
        ratios[ratios <= 0] = np.inf
        pivot_row = np.argmin(ratios) + 1

        tableau[pivot_row, :] /= tableau[pivot_row, pivot_col]
        for i in range(tableau.shape[0]):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

    x = np.zeros(num_vars)
    for i in range(num_vars):
        col = tableau[:, i]
        if col.sum() == 1 and len(col[col == 1]) == 1:
            row = np.where(col == 1)[0][0]
            x[i] = tableau[row, -1]

    z = tableau[0, -1]

    x1, x2 = x[0], x[1]
    return z, x1, x2

z, x1, x2 = simplex_algorithm(100, 150, 3, 2, 1, 2, 6, 5)
print(f"z: {z}, x1: {x1}, x2: {x2}")

z: 387.5, x1: 0.5, x2: 2.25


### Part2

In [22]:
import numpy as np

def simplex_algorithm(c1, c2, a11, a12, a21, a22, b1, b2):
    c = np.array([c1, c2])
    A = np.array([[a11, a12], [a21, a22]])
    b = np.array([b1, b2])

    num_vars = 2
    num_constraints = 2

    tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))

    tableau[0, :num_vars] = -c
    tableau[1:, :num_vars] = A
    tableau[1:, -1] = b

    tableau[1, num_vars] = 1
    tableau[2, num_vars + 1] = 1

    while True:
        pivot_col = np.argmin(tableau[0, :-1])
        if tableau[0, pivot_col] >= 0:
            break

        ratios = tableau[1:, -1] / tableau[1:, pivot_col]
        ratios[ratios <= 0] = np.inf
        pivot_row = np.argmin(ratios) + 1

        tableau[pivot_row, :] /= tableau[pivot_row, pivot_col]
        for i in range(tableau.shape[0]):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

    x = np.zeros(num_vars)
    for i in range(num_vars):
        col = tableau[:, i]
        if col.sum() == 1 and len(col[col == 1]) == 1:
            row = np.where(col == 1)[0][0]
            x[i] = tableau[row, -1]

    z = tableau[0, -1]
    return z, x[0], x[1]

def branch_and_bound_integer_solution(c1, c2, a11, a12, a21, a22, b1, b2):
    simplex_solution = simplex_algorithm(c1, c2, a11, a12, a21, a22, b1, b2)
    z, x1, x2 = simplex_solution

    def objective_function(x1, x2):
        return c1*x1 + c2*x2

    def constraint1(x1, x2):
        return a11*x1 + a12*x2 <= b1

    def constraint2(x1, x2):
        return a21*x1 + a22*x2 <= b2

    candidates = [
        (np.floor(x1), np.floor(x2)),
        (np.ceil(x1), np.floor(x2)),
        (np.floor(x1), np.ceil(x2)),
        (np.ceil(x1), np.ceil(x2))
    ]

    feasible_solutions = []
    for candidate_x1, candidate_x2 in candidates:
        if constraint1(candidate_x1, candidate_x2) and constraint2(candidate_x1, candidate_x2):
            z = objective_function(candidate_x1, candidate_x2)
            feasible_solutions.append((z, candidate_x1, candidate_x2))

    best_solution = max(feasible_solutions, key=lambda x: x[0]) if feasible_solutions else None

    return best_solution

branch_and_bound_integer_solution(100, 150, 3, 2, 1, 2, 6, 5)


(300.0, 0.0, 2.0)

### Part3

In [10]:
from scipy.optimize import linprog


In [14]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 5.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 6.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyabsa 2.3.4 requires protobuf<4.0.0, but you have protobuf 4.25.2 which is incompatible.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.2

In [15]:
from ortools.linear_solver import pywraplp

In [30]:
def solve_transportation_problem(capacities, demands, costs):
    
    solver = pywraplp.Solver.CreateSolver('SCIP')

    if not solver:
        return None

    num_sources = len(capacities)
    num_destinations = len(demands)

    x = {}
    for i in range(num_sources):
        for j in range(num_destinations):
            x[i, j] = solver.IntVar(0, solver.infinity(), f'x[{i},{j}]')

    objective = solver.Objective()
    for i in range(num_sources):
        for j in range(num_destinations):
            objective.SetCoefficient(x[i, j], costs[i][j])
    objective.SetMinimization()

    for j in range(num_destinations):
        demand_constraint = solver.Constraint(demands[j], demands[j])
        for i in range(num_sources):
            demand_constraint.SetCoefficient(x[i, j], 1)

    for i in range(num_sources):
        capacity_constraint = solver.Constraint(0, capacities[i])
        for j in range(num_destinations):
            capacity_constraint.SetCoefficient(x[i, j], 1)

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', objective.Value())
    for i in range(num_sources):
        for j in range(num_destinations):
            print(f'Transportation {x[i, j].solution_value()} units from Warehouse {i + 1} to City {j + 1}')

capacities = [500, 1800]
demands = [600, 700, 300]

costs = [
    [2, 1.5, 10],
    [4, 3.5, 6]
]
solve_transportation_problem(capacities, demands, costs)


Total cost =  5650.0
Transportation 0.0 units from Warehouse 1 to City 1
Transportation 500.0 units from Warehouse 1 to City 2
Transportation 0.0 units from Warehouse 1 to City 3
Transportation 600.0 units from Warehouse 2 to City 1
Transportation 200.0 units from Warehouse 2 to City 2
Transportation 300.0 units from Warehouse 2 to City 3
